In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
fn = r"C:\Users\Mi\YandexDisk\Shared\2020 umalat\schedules\2020.12.04.json"

In [5]:
date = cast_datetime(os.path.splitext(os.path.basename(fn))[0])
date

datetime.datetime(2020, 12, 4, 0, 0)

In [7]:
values = db.session.query(SKU).all()
skus = pd.DataFrame(values, columns=['sku'])
skus['name'] = skus['sku'].apply(lambda sku:sku.name)
skus['percent'] = skus['sku'].apply(lambda sku:sku.boilings[0].percent)
skus.head()

,sku,name,percent
0,<SKU 1>,"Моцарелла ""Pretto"" (для бутербродов), 45%, 0,2 кг, т/ф, (9 шт)",2.7
1,<SKU 2>,"Моцарелла ""Pretto"", 45%, 1,2 кг, в/у",2.7
2,<SKU 3>,"Моцарелла ""Unagrande"", 45%, 0,12 кг, ф/п (кубики)",2.7
3,<SKU 4>,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",2.7
4,<SKU 5>,"Моцарелла (палочки), 45%, кг, пл/л",2.7


In [8]:
values = db.session.query(Boiling).all()
boilings = pd.DataFrame(values, columns=['boiling'])
boilings['percent'] = boilings['boiling'].apply(lambda b: b.percent)
boilings

,boiling,percent
0,<Boiling 1>,2.7
1,<Boiling 2>,2.7
2,<Boiling 3>,2.7
3,<Boiling 4>,2.7
4,<Boiling 5>,3.3
5,<Boiling 6>,3.3
6,<Boiling 7>,3.3
7,<Boiling 8>,3.3
8,<Boiling 9>,3.6
9,<Boiling 10>,3.6


In [12]:
fn = r"C:\Users\Mi\YandexDisk\Shared\2020 umalat\schedules\2020.12.04.xlsx"

In [22]:
import openpyxl
wb = openpyxl.load_workbook(filename=fn, data_only=True)
ws = wb['планирование суточное']
values = []
for i in range(1, 200):
    if ws.cell(i, 3).value:
        values.append([ws.cell(i, j).value for j in range(3, 7)])

In [24]:
df = pd.DataFrame(values[1:], columns=['sku', 'remanings - request', 'normative remanings', 'plan']) # first value is header
df = df.fillna(0)
df = df[df['plan'] != 0]
df.head()

,sku,remanings - request,normative remanings,plan
0,"Моцарелла Фиор ди латте в воде ""Unagrande"", 50%, 0,125 кг, ф/п, (8 шт)",-1713,0,-1713.0
1,"Моцарелла Грандиоза в воде ""Unagrande"", 50%, 0,2 кг, ф/п",-33.6,0,-33.6
4,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, 0,125 кг, ф/п, (8 шт)",-100,0,-100.0
5,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, 0,1 кг, ф/п, (8 шт)",-192.8,0,-192.8
6,"Моцарелла Фиор ди Латте в воде ""Ваш выбор"", 50%, 0,1 кг, ф/п",-140.4,0,-140.4


In [26]:
sku_plan_df = df[['sku', 'plan']]
sku_plan_df['sku'] = sku_plan_df['sku'].apply(cast_sku)
sku_plan_df.head()

<ipython-input-26-128007e97e5b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_plan_df['sku'] = sku_plan_df['sku'].apply(cast_sku)


,sku,plan
0,<SKU 25>,-1713.0
1,<SKU 12>,-33.6
4,<SKU 24>,-100.0
5,<SKU 23>,-192.8
6,<SKU 26>,-140.4


# Genrate Json (deprecated)

In [15]:
from app.schedule_maker.models import *
df['sku'] = df[0].apply(lambda obj: db.session.query(SKU).filter(SKU.name == obj).first())
df = df[~df['sku'].isnull()]
df['boiling_type'] = df['sku'].apply(lambda sku: ' '.join([str(sku.boilings[0].percent), str(sku.boilings[0].ferment)]))
df['form_factor'] = df['sku'].apply(lambda sku: sku.form_factor.name)
df['sample_boiling'] = df['sku'].apply(lambda sku: sku.boilings[0])
df['sample_boiling_id'] = df['sku'].apply(lambda sku: sku.boilings[0].id)
df['size'] = df['sku'].apply(lambda sku: sku.weight_form_factor)
df.head()

,0,1,2,3,sku,boiling_type,form_factor,sample_boiling,sample_boiling_id,size
0,"Моцарелла Фиор ди латте в воде ""Unagrande"", 50%, 0,125 кг, ф/п, (8 шт)",-1713,0,-1713.0,<SKU 25>,3.6 Альче,Фиор Ди Латте,<Boiling 9>,9,125.0
1,"Моцарелла Грандиоза в воде ""Unagrande"", 50%, 0,2 кг, ф/п",-33.6,0,-33.6,<SKU 12>,3.6 Альче,Фиор Ди Латте,<Boiling 9>,9,200.0
4,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, 0,125 кг, ф/п, (8 шт)",-100,0,-100.0,<SKU 24>,3.3 Сакко,Фиор Ди Латте,<Boiling 6>,6,125.0
5,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, 0,1 кг, ф/п, (8 шт)",-192.8,0,-192.8,<SKU 23>,3.3 Сакко,Фиор Ди Латте,<Boiling 6>,6,100.0
6,"Моцарелла Фиор ди Латте в воде ""Ваш выбор"", 50%, 0,1 кг, ф/п",-140.4,0,-140.4,<SKU 26>,3.3 Сакко,Фиор Ди Латте,<Boiling 6>,6,100.0


In [42]:
# generate js
js = {'Date': date, 'WeekDay': date.weekday(), 'Boilings': []} 

for ff, ff_grp in df[df[3] != 0].groupby('boiling_type'):
    
    for i, (b, b_grp) in enumerate(ff_grp.groupby('sample_boiling_id')):
        df1 = b_grp.sort_values(by='size', ascending=False)
        values = {}
        for i, row in df1.iterrows():
            values[row['sku'].id] = abs(row[3])
        
        boiling_count = sum(values.values()) / 850 if '2.7' in row['boiling_type'] else sum(values.values()) / 1000
        boiling_count = math.ceil(boiling_count)
        
        js['Boilings'].append({'BoilingId': row['sample_boiling_id'],
                              'BoilingCount': boiling_count, 
                            'BoilingCountFactual': boiling_count,
                              'SKUVolumes': values,
                              'BoilingType': row['boiling_type'],
                              'TotalVolume': sum(values.values())})

In [43]:
js

{'Date': datetime.datetime(2020, 12, 4, 0, 0),
 'WeekDay': 4,
 'Boilings': [{'BoilingId': 1,
   'BoilingCount': 9,
   'BoilingCountFactual': 9,
   'SKUVolumes': {14: 800.0,
    34: 700.0,
    39: 1602.7200000000003,
    35: 907.52,
    17: 850.0,
    37: 848.4000000000001,
    4: 1758.0},
   'BoilingType': '2.7 Альче',
   'TotalVolume': 7466.639999999999},
  {'BoilingId': 2,
   'BoilingCount': 4,
   'BoilingCountFactual': 4,
   'SKUVolumes': {2: 1814.3999999999996, 16: 150.0, 1: 850.0},
   'BoilingType': '2.7 Сакко',
   'TotalVolume': 2814.3999999999996},
  {'BoilingId': 5,
   'BoilingCount': 1,
   'BoilingCountFactual': 1,
   'SKUVolumes': {21: 28.0},
   'BoilingType': '3.3 Альче',
   'TotalVolume': 28.0},
  {'BoilingId': 6,
   'BoilingCount': 2,
   'BoilingCountFactual': 2,
   'SKUVolumes': {24: 100.0,
    27: 329.0,
    22: 60.0,
    28: 24.0,
    23: 192.80000000000007,
    26: 140.4,
    45: 153.6,
    46: 153.6,
    29: 155.29999999999973,
    31: 210.0,
    32: 147.0},
   'Boili

# Function

In [44]:
def parse_schedule(fn):
    wb = openpyxl.load_workbook(filename=fn, data_only=True)
    ws = wb['планирование суточное']
    values = []
    for i in range(1, 200):
        if ws.cell(i, 3).value:
            values.append([ws.cell(i, j).value for j in range(3, 7)])
            
    df = pd.DataFrame(values[1:]) # first value is header
    df = df.fillna(0)
    df = df[df[3] != 0]
    df.head()
    
    df['sku'] = df[0].apply(lambda obj: db.session.query(SKU).filter(SKU.name == obj).first())
    df = df[~df['sku'].isnull()]
    df['boiling_type'] = df['sku'].apply(lambda sku: ' '.join([str(sku.boilings[0].percent), str(sku.boilings[0].ferment)]))
    df['form_factor'] = df['sku'].apply(lambda sku: sku.form_factor.name)
    df['sample_boiling'] = df['sku'].apply(lambda sku: sku.boilings[0])
    df['sample_boiling_id'] = df['sku'].apply(lambda sku: sku.boilings[0].id)
    df['size'] = df['sku'].apply(lambda sku: sku.weight_form_factor)
    df.head()
    
        # generate js
    js = {'Date': date, 'WeekDay': date.weekday(), 'Boilings': []} 

    for ff, ff_grp in df[df[3] != 0].groupby('boiling_type'):

        for i, (b, b_grp) in enumerate(ff_grp.groupby('sample_boiling_id')):
            df1 = b_grp.sort_values(by='size', ascending=False)
            values = {}
            for i, row in df1.iterrows():
                values[row['sku'].id] = abs(row[3])

            boiling_count = sum(values.values()) / 850 if '2.7' in row['boiling_type'] else sum(values.values()) / 1000
            boiling_count = math.ceil(boiling_count)

            js['Boilings'].append({'BoilingId': row['sample_boiling_id'],
                                  'BoilingCount': boiling_count, 
                                'BoilingCountFactual': boiling_count,
                                  'SKUVolumes': values,
                                  'BoilingType': row['boiling_type'],
                                  'TotalVolume': sum(values.values())})
            
    return js

# Todo: dump csvs instead of json

In [58]:
for fn in glob.glob(r'C:\Users\Mi\Desktop\umalat\schedules\*'):
    dirname, basename = os.path.dirname(fn), os.path.basename(fn)
    base, ext = os.path.splitext(basename)
    try:
        date = cast_datetime(base)
        new_fn = os.path.join(dirname, cast_str(date, '%Y.%m.%d') + ext)
        if os.path.abspath(fn) != os.path.abspath(new_fn):
            print('Renaming', fn, new_fn)
            rename(fn, new_fn)
    except:
        print(fn)

Renaming C:\Users\Mi\Desktop\umalat\schedules\05.12.20.xlsx C:\Users\Mi\Desktop\umalat\schedules\2020.12.05.xlsx
Renaming C:\Users\Mi\Desktop\umalat\schedules\06.12.20.xlsx C:\Users\Mi\Desktop\umalat\schedules\2020.12.06.xlsx
Renaming C:\Users\Mi\Desktop\umalat\schedules\08.12.2020.xlsx C:\Users\Mi\Desktop\umalat\schedules\2020.12.08.xlsx


In [ ]:
fns = glob.glob(r'C:\Users\Mi\Desktop\umalat\schedules\*.xlsx')
fns = [fn for fn in fns if '$' not in fn]

for fn in tqdm(fns):
    json_fn = os.path.splitext(fn)[0] + '.json'
    if os.path.exists(json_fn):
        continue
    print('Processing', fn)
    
    try:
        request = parse_schedule(fn)
        with open(json_fn, 'w', encoding='utf-8') as f:
            f.write(cast_js(request, indent=1))
    except:
        print('Failed to process', fn)
        print('Creating a stub')
        
        touch(json_fn)

Processing C:\Users\Mi\Desktop\umalat\schedules\2020.12.05.xlsx
